In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [2]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [3]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [7]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [8]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [9]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [10]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [11]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2091\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1781\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 628\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 179"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [12]:
def doGrade(grade):
    link = 'https://www.nationsreportcard.gov/''Dataservice/GetAdhocData.aspx?''type=data&subject=writing&grade=' + str(grade) + '&''subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'
    naep_resp1 = requests.get(link)
    print(naep_resp1)
    print(type(naep_resp1))
    try:
        results = naep_resp1.json()['result']
        return results
    except Exception as e:
        print('Failed to get result from API due to error:')
        print(e) # or just: pass
doGrade(4)
doGrade(8)
doGrade(12)

<Response [200]>
<class 'requests.models.Response'>
Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)
<Response [200]>
<class 'requests.models.Response'>


[{'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 2,
  'CohortLabel': 'Grade 8',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 8,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '1',
  'varValueLabel': 'Male',
  'value': 139.099504632971,
  'isStatDisplayable': 1,
  'errorFlag': 0},
 {'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 2,
  'CohortLabel': 'Grade 8',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 8,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '2',
  'varValueLabel': 'Female',
  'value': 158.567104984955,
  'isStatDisplayable': 1,
  'errorFlag': 0}]

<Response [200]>
<class 'requests.models.Response'>


[{'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 3,
  'CohortLabel': 'Grade 12',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 12,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '1',
  'varValueLabel': 'Male',
  'value': 141.256977963264,
  'isStatDisplayable': 1,
  'errorFlag': 0},
 {'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 3,
  'CohortLabel': 'Grade 12',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 12,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '2',
  'varValueLabel': 'Female',
  'value': 155.385916780351,
  'isStatDisplayable': 1,
  'errorFlag': 0}]

# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [13]:
## get the key
API_KEY = "aNnSXLscmWZ838q4i6_es1vOMqXkJeVX1AtzyCe-akao5XpoNlZzKasdrL3coPpe5007iWUpQXF4P2vpF56MZjPYoiGVUkcNlcYiOUuXKslM9sNkroKCC9RQ3qszZnYx"

In [23]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Bethesda,MD"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [24]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'exLT5_TgGf4AvYGwz98S-A',
 'alias': 'aventino-bethesda-2',
 'name': 'Aventino',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/GBh5RPTX0K6Xl6PGsi9fjg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/aventino-bethesda-2?adjust_creative=-9b9vG2tOg8KCdaop5P6GQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-9b9vG2tOg8KCdaop5P6GQ',
 'review_count': 27,
 'categories': [{'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.4,
 'coordinates': {'latitude': 38.981252, 'longitude': -77.094493},
 'transactions': [],
 'price': '$$$',
 'location': {'address1': '4747 Bethesda Ave',
  'address2': '',
  'address3': None,
  'city': 'Bethesda',
  'zip_code': '20814',
  'country': 'US',
  'state': 'MD',
  'display_address': ['4747 Bethesda Ave', 'Bethesda, MD 20814']},
 'phone': '+13019616450',
 'display_phone': '(301) 961-6450',
 'distance': 1830.8798835265868,
 'attributes': {'business_temp_closed': None,
  'menu_url': 'https://www.aventinocucina.co

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,attributes
0,exLT5_TgGf4AvYGwz98S-A,aventino-bethesda-2,Aventino,https://s3-media3.fl.yelpcdn.com/bphoto/GBh5RP...,False,https://www.yelp.com/biz/aventino-bethesda-2?a...,27,"[{'alias': 'italian', 'title': 'Italian'}]",4.4,"{'latitude': 38.981252, 'longitude': -77.094493}",[],$$$,"{'address1': '4747 Bethesda Ave', 'address2': ...",+13019616450,(301) 961-6450,1830.879884,"{'business_temp_closed': None, 'menu_url': 'ht..."
1,CU9E8nMBEJLoDHzUzaz-ig,dig-coming-soon-bethesda,Dig,https://s3-media2.fl.yelpcdn.com/bphoto/ZlrWyK...,False,https://www.yelp.com/biz/dig-coming-soon-bethe...,26,"[{'alias': 'newamerican', 'title': 'New Americ...",4.3,"{'latitude': 38.9824019, 'longitude': -77.0956...","[delivery, pickup]",NaN,"{'address1': '4733 Elm St', 'address2': None, ...",+13019683636,(301) 968-3636,1668.229334,"{'business_temp_closed': None, 'menu_url': 'ht..."
2,d3Xmu3igy9yv5cFsBkNxeQ,ala-bethesda-bethesda,Ala - Bethesda,https://s3-media2.fl.yelpcdn.com/bphoto/PhuQN_...,False,https://www.yelp.com/biz/ala-bethesda-bethesda...,20,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 38.98740950671368, 'longitude': -...",[],NaN,"{'address1': '4948 Fairmont Ave', 'address2': ...",+12027171166,(202) 717-1166,1196.966743,"{'business_temp_closed': None, 'menu_url': 'ht..."
3,6vPNPahPY9CFjqpYAgvxww,barrel-and-crow-bethesda-2,Barrel & Crow,https://s3-media4.fl.yelpcdn.com/bphoto/8FIpZ1...,False,https://www.yelp.com/biz/barrel-and-crow-bethe...,539,"[{'alias': 'southern', 'title': 'Southern'}, {...",4.0,"{'latitude': 38.9897544133054, 'longitude': -7...",[delivery],$$,"{'address1': '4867 Cordell Ave', 'address2': '...",+12408003253,(240) 800-3253,1026.633071,"{'business_temp_closed': None, 'menu_url': 'ht..."
4,f6gw5tzCTq4pGa2aBPJM4A,hawkers-asian-street-food-bethesda,Hawkers Asian Street Food,https://s3-media2.fl.yelpcdn.com/bphoto/PsVFPV...,False,https://www.yelp.com/biz/hawkers-asian-street-...,835,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.2,"{'latitude': 38.981561, 'longitude': -77.097674}",[],$$,"{'address1': '7117 Bethesda Ln', 'address2': N...",+12402038135,(240) 203-8135,1608.559749,"{'business_temp_closed': None, 'menu_url': 'ht..."


In [22]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
YlTyBzagifcasFA0n8OpkQ,YlTyBzagifcasFA0n8OpkQ,金峰魯肉飯-中正區,Jin Feng Braised Pork Rice,https://s3-media4.fl.yelpcdn.com/bphoto/90r5r4...,https://www.yelp.com/biz/%E9%87%91%E5%B3%B0%E9...,$,+886223960808,+886 2 2396 0808
8Qbbb9EaqdTaAI09WQuIzA,8Qbbb9EaqdTaAI09WQuIzA,阜杭豆漿-中正區,Fuhang Soy Milk,https://s3-media3.fl.yelpcdn.com/bphoto/zi5tIk...,https://www.yelp.com/biz/%E9%98%9C%E6%9D%AD%E8...,$,+886223922175,+886 2 2392 2175
5gNHAIyajmJ_4i5vXwOVRw,5gNHAIyajmJ_4i5vXwOVRw,永和豆漿大王-大安區-2,Yong He Soy Milk King,https://s3-media2.fl.yelpcdn.com/bphoto/g_CeWw...,https://www.yelp.com/biz/%E6%B0%B8%E5%92%8C%E8...,$,+886227035051,+886 2 2703 5051
0ccCDXUfKrwoU1ciD2CIew,0ccCDXUfKrwoU1ciD2CIew,萬華陳記專業腸蚵麵線-萬華區,萬華陳記專業腸蚵麵線,https://s3-media2.fl.yelpcdn.com/bphoto/GsS6qu...,https://www.yelp.com/biz/%E8%90%AC%E8%8F%AF%E9...,$,+886223041979,+886 2 2304 1979
Xbmdhb5PEfbAPNxzghWCRQ,Xbmdhb5PEfbAPNxzghWCRQ,上引水產-中山區,Addiction Aquatic Development,https://s3-media4.fl.yelpcdn.com/bphoto/tvdcWp...,https://www.yelp.com/biz/%E4%B8%8A%E5%BC%95%E6...,$$,+886225081268,+886 2 2508 1268
Smu2CQeGeGnPnqxVmaAejg,Smu2CQeGeGnPnqxVmaAejg,湯包-萬華區,Soup Dumplings,https://s3-media3.fl.yelpcdn.com/bphoto/QkfgdC...,https://www.yelp.com/biz/%E6%B9%AF%E5%8C%85-%E...,NaN,+886223717762,+886 2 2371 7762
TexN3c8guzJoyOVdD5kAOA,TexN3c8guzJoyOVdD5kAOA,原西園橋下-萬華區,原西園橋下,https://s3-media4.fl.yelpcdn.com/bphoto/6VLb9h...,https://www.yelp.com/biz/%E5%8E%9F%E8%A5%BF%E5...,NaN,,


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [25]:
def get_business_reviews(business_ids):
    base_url = "https://api.yelp.com/v3/businesses/{}/reviews"
    header = {'Authorization': f"Bearer {API_KEY}"}
    total_reviews = []
    
    for bus_id in business_ids:
        total_query = base_url.format(bus_id)
        yelp_genresp = requests.get(total_query, headers = header)
        yelp_genresp
        yelp_genjson = yelp_genresp.json()['reviews']
        yelp_genjson
        for rev_dict in yelp_genjson:
            rev_dict["business_id"] = bus_id
        total_reviews += yelp_genjson
    final_df = pd.DataFrame(total_reviews)
    return final_df


get_business_reviews(["MRTdhLfHe-Uw-qQnQXxgVg", "PqyJx0ndtwBT9wLeFYnhoQ", "zeFASIGTjBwwPNf0ab86Jw"])

KeyError: 'reviews'